In [49]:
import numpy as np
import math
from numba import njit, types, prange


In [77]:
def _funtext_1d(name:str ,expr:str, dict) -> str:
    lines = [
        f"def {name}(x, r, param):",
    ]
    for i, (key, value) in enumerate(dict.items()):
        if not isinstance(key, str):
            raise TypeError(f"Only str keys supported, got {type(key)!r}")
        if not key.isidentifier():
            raise ValueError(f"Key {key!r} is not a valid Python identifier")
        lines.extend([
        f"    {key} = param[{i!r}]"
        ])
    lines.extend([
        f"    x_next = {expr}",
        f"    return x_next"
    ])
    source = "\n".join(lines)
    return source

def _funtext_2d_step(name: str, expr_x: str, expr_y: str, dict) -> str:
    lines = [
        f"def {name}(x, y, r, s, param):",
    ]
    for i, (key, value) in enumerate(dict.items()):
        if not isinstance(key, str):
            raise TypeError(f"Only str keys supported, got {type(key)!r}")
        if not key.isidentifier():
            raise ValueError(f"Key {key!r} is not a valid Python identifier")
        lines.extend([
        f"    {key} = param[{i!r}]"
        ])
    lines.extend([
        f"    x_next = {expr_x}",
        f"    y_next = {expr_y}",
        f"    return x_next, y_next",
    ])
    source = "\n".join(lines)
    return source

def _funtext_2d_jac(
     name: str, dXdx: str, dXdy: str, dYdx: str, dYdy: str, dict
) -> str:
    lines = [
        f"def {name}(x, y, r, s, param):",
    ]
    for i, (key, value) in enumerate(dict.items()):
        if not isinstance(key, str):
            raise TypeError(f"Only str keys supported, got {type(key)!r}")
        if not key.isidentifier():
            raise ValueError(f"Key {key!r} is not a valid Python identifier")
        lines.extend([
        f"    {key} = param[{i!r}]"
        ])
    lines.extend([
        f"    dxdx = {dXdx}",
        f"    dxdy = {dXdy}",
        f"    dydx = {dYdx}",
        f"    dydy = {dYdy}",
        f"    return dxdx, dxdy, dydx, dydy",
    ])
    source = "\n".join(lines)
    return source



In [78]:
cfg = {
    "alpha": 0.5, 
    "beta":1.2, 
    "zeta":4.0
}

In [79]:
print(_funtext_1d("fun1","a*r*(1-r)",cfg))
print(list(cfg.values()))

def fun1(x, r, param):
    alpha = param[0]
    beta = param[1]
    zeta = param[2]
    x_next = a*r*(1-r)
    return x_next
[0.5, 1.2, 4.0]


In [35]:
print(_funtext_2d_step("fun1","a*x*(1-y)","a*y*(1-x)",cfg))

def fun1(x, y, r, s, param):
    a = param[0]
    b = param[1]
    c = param[2]
    d = param[3]
    alpha = 0.5
    beta = 1.2
    zeta = 4.0
    x_next = a*x*(1-y)
    y_next = a*y*(1-x)
    return x_next, y_next


In [36]:
@njit(types.float64(types.float64), cache=True, fastmath=False)
def DiracDelta(x):
    # We ignore distributional spikes; enough for Lyapunov purposes.
    return 0.0

@njit(types.float64(types.float64), cache=True, fastmath=False)
def Heaviside(x):
    return 1.0 if x > 0.0 else 0.0

@njit(types.float64(types.float64), cache=True, fastmath=False)
def step(x):
    return 1.0 if x > 0.0 else 0.0

@njit(types.float64(types.float64), cache=True, fastmath=False)
def sign(x):
    return 1.0 if x > 0.0 else -1.0

@njit(types.float64(types.float64), cache=True, fastmath=False)
def Abs(x):
    return np.abs(x)

@njit(types.float64(types.float64), cache=True, fastmath=False)
def re(x):
    return x

@njit(types.float64(types.float64), cache=True, fastmath=False)
def im(x):
    return 0.0

@njit(types.float64(types.float64), cache=True, fastmath=False)
def sec(x):
    return 1.0 / np.cos(x)

@njit(types.float64(types.float64), cache=True, fastmath=False)
def mod1(x):
    return x % 1.0

@njit(types.float64(types.float64, types.float64), cache=True, fastmath=False)
def Mod(x, v):
    return x % v

@njit(types.float64(types.float64, types.float64), cache=True, fastmath=False)
def Derivative(x, v):
    # never actually used; placeholder to keep SymPy happy if it
    # sneaks in.
    return 1.0

@njit(types.float64(types.float64, types.float64), cache=True, fastmath=False)
def apow(x, a):
    return np.sign(x)*np.pow(np.abs(x),a)

@njit(types.float64(types.float64), cache=True, fastmath=False)
def floor(x):
    return math.floor(x)

@njit(types.float64(types.float64), cache=True, fastmath=False)
def ceil(x):
    return math.ceil(x)

In [37]:
def _make_1d_step_func(expr: str, dict):
    """
    Create a plain Python function f(x, r, a) that can be jitted.
    """
    ns = {
        "step": step,
        "Heaviside": Heaviside,
        "DiracDelta": DiracDelta,
        "sign": sign,
        "Abs": Abs,
        "abs": Abs,
        "sin": np.sin,
        "cos": np.cos,
        "tan": np.tan,
        "sec": sec,
        "cosh": np.cosh,
        "sinh": np.sinh,
        "exp": np.exp,
        "pow": np.power,
        "apow": apow,
        "log": np.log,
        "mod1": mod1,
        "Mod": Mod,
        "Derivative": Derivative,
        "re": re,
        "im": im,
        "pi": np.pi,
        "np": np,
        "math": math,
        "max": max,
        "min": min,
        "floor": floor,
        "ceil": ceil,
    }
    src = _funtext_1d_step("impl",expr, dict)
    exec(src, ns, ns)
    return ns["impl"]

In [38]:
cfg = dict(alpha=0.5,beta=1.2,zeta=4.0)
f1 = _make_1d_step_func("r*x*(1-x)",cfg)
x=0.5
for i in range(5):
    x=f1(x,0.5,np.array([0,0,0,0]))
    print(f"x: {x}")


x: 0.125
x: 0.0546875
x: 0.025848388671875
x: 0.012590124737471342
x: 0.006215806748283127


In [39]:
# 1d step sig
STEP_SIG = types.float64(types.float64, types.float64, types.float64[:])
step_py = _make_1d_step_func("r*x*(1-x)",cfg)
step_jit = njit(STEP_SIG, cache=False, fastmath=False)(step_py)
x=float(0.5)
r=float(0.5)
for i in range(5):
    x=step_jit(x,r,np.asarray([0,0,0,0],dtype=np.float64,))
    print(f"x: {x}")



x: 0.125
x: 0.0546875
x: 0.025848388671875
x: 0.012590124737471342
x: 0.006215806748283127


In [40]:
x=[
    "aaaa",
    "bbbb",
    "ccccc"
]

x.append([
    "zzzz",
    "yyyy",
    "xxxx",
    "wwwww",
    "vvvvvv"
])

print(x)

['aaaa', 'bbbb', 'ccccc', ['zzzz', 'yyyy', 'xxxx', 'wwwww', 'vvvvvv']]


In [48]:
cfg = {
    "alpha": 0.5, 
    "beta":1.2, 
    "zeta":4.0
}

np.array(list(cfg.values()),dtype=np.float64)

list(cfg.keys())

for i, (key, value) in enumerate(cfg.items()):
    print(f"{i} {key} {value}")


0 alpha 0.5
1 beta 1.2
2 zeta 4.0


In [73]:
cfg = {
    "alpha": 0.5, 
    "beta":1.2, 
    "zeta":4.0
}

lines=[]
for i, (key, value) in enumerate(cfg.items()):
    print(f"{i} {key} {value}")
    lines.append(f" {key}={value!r}")

lines.extend(["aaaaa","sssssss","eeeeeee"])
lines

0 alpha 0.5
1 beta 1.2
2 zeta 4.0


[' alpha=0.5', ' beta=1.2', ' zeta=4.0', 'aaaaa', 'sssssss', 'eeeeeee']

In [82]:
cfg = {
    "alpha": 0.5, 
    "beta":1.2, 
    "zeta":4.0
}

list(cfg.keys()).index('beta')

1

In [85]:
x="aaaa"
f"x={x}"

'x=aaaa'

In [87]:
import sys
sys.path.insert(0, "/Users/nicknassuphis")
import time
import math
import argparse
import re as regex
import numpy as np
import sympy as sp
from numba import njit, types, prange

from specparser import specparser, expandspec
from rasterizer import raster
from rasterizer import colors

specparser.split_chain("a:1:2,b,c:1:2")



{'a': ['1', '2'], 'b': [], 'c': ['1', '2']}